In [1]:
from argparse import ArgumentParser
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from torch.utils.data import random_split
from torchvision import transforms
import os
from knn_cuda import KNN 
import pointnet2_ops._ext as _ext
import numpy as np
import pickle
from tqdm import tqdm

from lit_dataset_clean import HumanDataModule

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pickle

with open('args.pkl', 'rb') as f:
    args = pickle.load(f)

dm_human = HumanDataModule
dm_human = dm_human.from_argparse_args(args) 
self = dm_human

In [3]:
args.input_pts = 1024
args.batch_size=2
from lit_corrnet3d_clean import LitCorrNet3D

model = LitCorrNet3D(**vars(args))
logger = pl.loggers.TensorBoardLogger("tb_logs", name="my_model")
trainer = pl.Trainer.from_argparse_args(args, gpus=str(args.gpus), benchmark=True, deterministic=True, logger=logger) #gpu
model.hparams.lr = 0.02089296130854041

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: You requested multiple GPUs but did not specify a backend, e.g. `Trainer(accelerator="dp"|"ddp"|"ddp2")`. Setting `accelerator="ddp_spawn"` for you.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


In [4]:
train_loader = dm_human.train_dataloader()
batch = next(iter(train_loader))

In [5]:
label, pinput1, input2, index_ = batch
label = label.cuda()
pinput1 = pinput1.cuda()
input2 = input2.cuda()
index_ = index_.cuda()
model = model.cuda()
label.shape, pinput1.shape, input2.shape, index_.shape


(torch.Size([8, 1024, 1024]),
 torch.Size([8, 1024, 3]),
 torch.Size([8, 1024, 3]),
 torch.Size([8]))

In [6]:
self = model
p, out_a, out_b =self._run_step(pinput1,input2)
rec_term, rank_term, mfd_term = self._run_loss(pinput1.transpose(2, 1), input2.transpose(2, 1), p, out_a, out_b)

In [8]:
self.rec_coeff, self.rank_coeff, self.mfd_coeff

(10.0, 1.0, 0.1)

In [7]:
p.shape, out_a.shape, out_b.shape

(torch.Size([8, 1024, 1024]),
 torch.Size([8, 3, 1024]),
 torch.Size([8, 3, 1024]))

In [11]:
p.sum(axis=-1)[0]

tensor([0.2822, 1.9973, 1.1938,  ..., 0.0396, 2.1568, 0.0599], device='cuda:0',
       grad_fn=<SelectBackward>)

In [38]:
# cross entropy loss between p and label
loss = F.cross_entropy(p.transpose(1, 2), torch.max(label.transpose(1, 2), dim=-1).indices)
loss

tensor(6.9319, device='cuda:0', grad_fn=<NllLoss2DBackward>)

In [13]:
p[0][7][17], p[0][17][7]

(tensor(3.4444e-14, device='cuda:0', grad_fn=<SelectBackward>),
 tensor(7.9156e-08, device='cuda:0', grad_fn=<SelectBackward>))

In [19]:
# find the column where p is 1 for each row
label.argmax(dim=0)

tensor([[7, 7, 7,  ..., 7, 7, 7],
        [7, 7, 7,  ..., 7, 7, 7],
        [7, 7, 7,  ..., 7, 7, 7],
        ...,
        [7, 7, 7,  ..., 7, 7, 7],
        [7, 7, 7,  ..., 7, 7, 7],
        [7, 7, 7,  ..., 7, 7, 7]], device='cuda:0')

In [14]:
p[0].argmax(dim=0)

tensor([ 524, 1017,  152,  ...,  752,  469,  108], device='cuda:0')

In [18]:
p[0].argmax(dim=0)

tensor([ 524, 1017,  152,  ...,  752,  469,  108], device='cuda:0')

In [35]:
torch.max(label.transpose(1, 2), dim=-1).indices[0]

tensor([608, 260, 365,  ..., 466, 538, 104], device='cuda:0')

In [37]:
torch.max(label, dim=-2).indices[0]

tensor([608, 260, 365,  ..., 466, 538, 104], device='cuda:0')